In [1]:
import os
import re
import pickle
import numpy as np
from dotenv import dotenv_values
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from tqdm import tqdm
from evaluate import load
import pandas as pd

In [2]:
# Load env file with API KEY using full path
config = dotenv_values(".env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]

In [3]:
topic_to_label = {
    "Music": ["Recording", "MusicRecording/name", "Duration", "ArtistName", "AlbumName"],
    "Restaurants" : ["RestaurantName", "PriceRange", "AddressRegion", "Country", "Telephone", 
                     "PaymentAccepted", "PostalCode",
                     "Coordinate", "DayOfWeek", "Time",
                     "RestaurantDescription", "Review"],
    "Hotels" : ["HotelName", "PriceRange", "Telephone",
                "FaxNumber", "Country", "Time",
                "PostalCode", "AddressLocality", "email",
                "LocationFeatureSpecification",
                "HotelDescription", "Review", "Rating",
                "PaymentAccepted", "Photograph"],
    "Events" : ["EventName", "Date", "DateTime",
                "EventStatusType", "EventDescription",
                "EventAttendanceModeEnumeration",
                "Organization", "Currency", "Telephone"]
}

In [4]:
labels_to_text = {
    "I dont'know": "I dont'know",
    "addressLocality": "locality of address",
    "postalCode": "postal code",
    "addressRegion": "region of address",
    "Country": "country",
    "priceRange": "price range",
    "Hotel/name": "name of hotel",
    "telephone": "telephone",
    "faxNumber": "fax number",
    "Date": "date",
    "Restaurant/name": "name of restaurant",
    "paymentAccepted": "payment accepted",
    "DayOfWeek": "day of week",
    "Review": "review",
    "Organization": "organization",
    "DateTime": "date and time",
    "MusicAlbum/name": "name of music album",
    "MusicArtistAT": "music artist",
    "MusicRecording/name": "name of music recording",
    "Photograph": "photograph",
    "CoordinateAT": "coordinate",
    "Event/name": "name of event",
    "EventAttendanceModeEnumeration": "event attendance mode",
    "EventStatusType": "event status",
    "currency": "currency",
    "email": "email",
    "Time": "time",
    "LocationFeatureSpecification": "location feature",
    "Duration": "duration",
    "Event/description": "description of event",
    "Restaurant/description": "description of restaurant",
    "Rating": "rating",
    "Hotel/description": "description of hotel"
}

In [5]:
# Dictionary to map ChatGPT answers to label set: synonyms can be added here
text_to_label = {
    "locality of address": "addressLocality",
    "postal code": "postalCode",
    "region of address": "addressRegion",
    "country": "Country",
    "price range": "priceRange",
    "name of hotel": "Hotel/name",
    "telephone": "telephone",
    "fax number": "faxNumber",
    "date": "Date",
    "name of restaurant": "Restaurant/name",
    "payment accepted": "paymentAccepted",
    "day of week": "DayOfWeek",
    "review": "Review",
    "organization": "Organization",
    "date and time": "DateTime",
    "music artist": "MusicArtistAT",
    "music album": "MusicAlbum/name",
    "name of music recording": "MusicRecording/name",
    "photograph": "Photograph",
    "coordinate": "CoordinateAT",
    "name of event": "Event/name",
    "event attendance mode": "EventAttendanceModeEnumeration",
    "event status": "EventStatusType",
    "currency": "currency",
    "email": "email",
    "time": "Time",
    "location feature": "LocationFeatureSpecification",
    "duration": "Duration",
    "description of event": "Event/description",
    "description of restaurant": "Restaurant/description",
    "description of hotel": "Hotel/description",
    "rating": "Rating",
    #Added
    "description of restaurants": "Restaurant/description",
    "name of music artist": "MusicArtistAT",
    "description of hotel amenities": "LocationFeatureSpecification",
    "amenities": "LocationFeatureSpecification",
    "name of album": "MusicAlbum/name",
    "i don't know": "-",
    "name of music album": "MusicAlbum/name",
    "music recording": "MusicRecording/name",
    "event name": "Event/name",
    "description of hotels": "Hotel/description",
    "name of hotels": "Hotel/name",
    "duration of music recording or video": "Duration",
    "name of organization": "Organization",
    "hotel amenities": "LocationFeatureSpecification",
    "amenities of hotel room": "LocationFeatureSpecification",
    "check-in time": "Time",
    "check-out time": "Time",
    "time of check-in": "Time",
    "time of check-out": "Time",
    "hotel features": "LocationFeatureSpecification",
    "name of aparthotel": "Hotel/name",
    "event description": "Event/description",
    "email address": "email",
    "room amenities": "LocationFeatureSpecification",
    "end date": "Date",
    "descriptions of events": "Event/description",
    "mode of attendance": "EventAttendanceModeEnumeration",
    "name of song": "MusicRecording/name"
}

topic_to_label = {
    "Music": ["name of music recording", "name of song", "music recording", "duration", "duration of music recording or video", 
              "music artist", "name of music artist", "music album", "name of album", "name of music album"],
    "Restaurants" : ["name of restaurant", "price range", "region of address", "country", "telephone", 
                     "payment accepted", "postal code",
                     "coordinate", "day of week", "time", "check-in time", "check-out time", "time of check-in", "time of check-out",
                     "description of restaurants", "description of restaurant", "review"],
    "Hotels" : ["HotelName", "PriceRange", "Telephone",
                "FaxNumber", "Country", "Time",
                "PostalCode", "AddressLocality", "email",
                "LocationFeatureSpecification",
                "HotelDescription", "review", "Rating",
                "payment accepted", "Photograph"],
    "Events" : ["EventName", "Date", "DateTime",
                "event status", "description of event", "description of events", "event description",
                "EventAttendanceModeEnumeration",
                "Organization", "Currency", "Telephone"]
}

## Load test set

In [6]:
with open('data/cta-test-table-wise.pkl', "rb") as f:
    test = pickle.load(f)

examples = [example[1] for example in test ]
labels = [l for example in test for l in example[2]]

In [7]:
print(len(examples))
print(len(labels))

41
250


In [8]:
topics = [example[3] for example in test]
print(topics)

['Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Restaurant', 'Event', 'Event', 'Event', 'Event', 'Event', 'Event', 'Event', 'Event', 'Event', 'Event', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'Hotel', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording', 'MusicRecording']


## Choose prompt template: without or with instructions

In [9]:
# New prompt
semantic_concept_template = """

Answer the question based on the task below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest a semantic concept for each column of a given table. Answer with the semantic concept for each column with the format Column1: semantic concept.

Table: {input}

Semantic concepts:

"""

In [10]:
# Paper name: table + instructions
inst_template = """

Answer the question based on the task and instructions below. If the question cannot be answered using the information provided answer with "I don't know".

Task: Suggest a semantic concept for each column of a given table.

Instructions: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, suggest a semantic concept that best represents the meaning of all cells in the column. 4. Answer with the semantic concept for each column with the format Column1: semantic concept.

Table:
{input}

Semantic concepts:

"""

In [11]:
# prompt to ask gpt to classify labels 
classify_label_template = """

Task: Classify the semantic concept {input} with only one of the following classes that are separated with comma: description of event, description of restaurant, locality of address, postal code, region of address, country, price range, telephone, date, name of restaurant, payment accepted, day of week, review, organization, date and time, coordinate, name of event, event attendance mode, event status, currency, time, description of hotel, name of hotel, location feature, rating, fax number, email, photograph, name of music recording, music artist, name of album, duration.

Class:

"""

## Load LLM and run model

In [ ]:
gpt_3_turbo = ChatOpenAI(model_name='gpt-3.5-turbo-0301', temperature=0)
gpt_4 = ChatOpenAI(model_name='gpt-4', temperature=0)

In [ ]:
prompt_type = "semantic_concept"
if prompt_type == "semantic_concept":
    prompt = PromptTemplate(template=semantic_concept_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)
elif prompt_type == "with_inst":
    prompt = PromptTemplate(template=inst_template, input_variables=['input'])
    llm_chain = LLMChain(prompt=prompt, llm=gpt_3_turbo)
    llm_chain_4 = LLMChain(prompt=prompt, llm=gpt_4)

In [12]:
def save_preds(preds: list[str], file_name: str):
    #Save predictions in a file:
    with open(file_name,'wb') as f:
        pickle.dump(preds,f)

def load_preds(file_name: str):
    #Save predictions in a file:
    with open(file_name,'rb') as f:
        preds = pickle.load(f)
    return preds

In [ ]:
#Zero-shot prediction
preds_gpt35 = [llm_chain.run({'input': example}) for example in examples]
save_preds(preds_gpt35, "predictions/gpt35-prompt-table-with-instructions.pkl")

In [ ]:
preds_gpt4 = [llm_chain_4.run({'input': example}) for example in examples]
save_preds(preds_gpt4, "predictions/gpt4-prompt-table-with-instructions.pkl")

In [85]:
preds = load_preds("predictions/gpt35-prompt-table-with-instructions.pkl")
print(preds)

['Column 1: Phone number\nColumn 2: Restaurant name\nColumn 3: Zip code\nColumn 4: State\nColumn 5: Country\nColumn 6: Longitude\nColumn 7: Latitude\nColumn 8: Operating hours\nColumn 9: Days of the week', 'Column 1: Phone number\nColumn 2: Restaurant name\nColumn 3: Restaurant description\nColumn 4: Location\nColumn 5: Operating days\nColumn 6: Operating hours', 'Column 1: Restaurant Name\nColumn 2: Postal Code\nColumn 3: Payment Methods\nColumn 4: State\nColumn 5: Suburb', 'Column 1: Restaurant Name\nColumn 2: Phone Number\nColumn 3: Menu Description\nColumn 4: City\nColumn 5: State\nColumn 6: Accepted Payment Methods\nColumn 7: Zip Code\nColumn 8: Longitude\nColumn 9: Latitude\nColumn 10: Opening Hours\nColumn 11: Days of the Week', 'Column 1: Restaurant Name\nColumn 2: Price Range\nColumn 3: Phone Number', 'Column 1: Restaurant name\nColumn 2: Phone number\nColumn 3: Price range\nColumn 4: Country code\nColumn 5: Department/region\nColumn 6: Postal code\nColumn 7: Longitude\nColumn

In [ ]:
df = pd.DataFrame(data={"raw_prompt_output": preds})
df.to_csv("predictions/gpt4-prompt-table-without-instructions.csv")

## Evaluation

In [ ]:
prompt_classify = PromptTemplate(template=classify_label_template, input_variables=['input'])
llm_chain_c = LLMChain(prompt=prompt_classify, llm=gpt_3_turbo)
llm_chain_4_c = LLMChain(prompt=prompt_classify, llm=gpt_4)

In [ ]:
llm_chain_c.run({'input': "state"})

In [14]:
bertscore = load("bertscore")
predictions = ["hello there", "hello there"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)

{'precision': [1.000000238418579, 0.8224340677261353], 'recall': [1.000000238418579, 0.8144457340240479], 'f1': [1.000000238418579, 0.81842041015625], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.9(hug_trans=4.24.0)'}


In [86]:
def evaluation_new(preds, bert_threshold=0.85, topics=None):
    ids, predictions, original_preds, top_5_predictions, highest_bertscores = [], [], [], [], []
    i=0
    for j, table_preds in enumerate(tqdm(preds)):
        # How many columns does the table have? : To control cases when less/more classes are returned
        table_number = len(test[j][2])
        
        if "Semantic concepts:" in table_preds:
            table_preds = table_preds.split("Class:")[1]
        
        #Break predictions into either \n or ,
        if ":" in table_preds or "-" in table_preds:
            if ":" in table_preds:
                separator = ":"
                start = 1
                end = table_number+1
            else:
                separator = "-"  
                start = 1
                end = table_number+1
        else:
            separator = ","
            start = 0
            end = table_number
            
        col_preds = table_preds.split(separator)[start:end]
        for pred in col_preds:
            i+=1
            
            # Remove break lines
            if "\n" in pred:
                pred = pred.split('\n')[0].strip()
            # Remove commas
            if "," in pred:
                pred = pred.split(",")[0].strip()
            # Remove paranthesis
            if '(' in pred:
                pred = pred.split("(")[0].strip()
            #Remove points
            if '.' in pred:
                pred = pred.split(".")[0].strip()
            # Lower-case prediction
            pred = pred.strip().lower()
            original_preds.append(pred)
            ids.append(j)

            classes = list(text_to_label.keys())
            bertscores = np.array(bertscore.compute(predictions=[pred] * len(classes), references=classes, lang="en")["f1"])
            index = np.argsort(bertscores)[-5:]
            max_index = index[-1]
            top_5_predictions.append([text_to_label[classes[i]] for i in index])
            highest_bertscores.append(bertscores[index])

            highest_score = bertscores[max_index]

            if highest_score > bert_threshold:
                predictions.append(text_to_label[classes[max_index]])
            else:
                print(f"For test example {i} out of label space prediction: {pred}")
                predictions.append('-')
            
        # If more/less predictions for table
        if len(col_preds) < table_number:
            for m in range(0, table_number-len(col_preds)):
                predictions.append('-')
                i+=1
    return ids, predictions, original_preds, highest_bertscores, top_5_predictions

ids, class_predictions, original_preds, highest_bertscores, top_5_preds = evaluation_new(preds)

 59%|█████▊    | 24/41 [03:34<02:53, 10.21s/it]

For test example 173 out of label space prediction: accommodation rating


100%|██████████| 41/41 [05:16<00:00,  7.71s/it]


In [87]:
df = pd.DataFrame({"prompt_output_id": ids, 
                   "label": labels, 
                   "parsed_col_pred": original_preds, 
                   "class_pred_using_bert": class_predictions, 
                   "highest_bertscore": highest_bertscores,
                   "top_5_preds": top_5_preds})
df.to_csv("predictions/preds_gpt35_with_inst_2.csv")

In [ ]:
# Map predictions to label space
def evaluation_old():
    predictions = []
    i=0
    for j, table_preds in enumerate(preds):
        # How many columns does the table have? : To control cases when less/more classes are returned
        table_number = len(test[j][2])
        
        if "Class:" in table_preds:
            table_preds = table_preds.split("Class:")[1]
        
        #Break predictions into either \n or ,
        if ":" in table_preds or "-" in table_preds:
            if ":" in table_preds:
                separator = ":"
                start = 1
                end = table_number+1
            else:
                separator = "-"  
                start = 1
                end = table_number+1
        else:
            separator = ","
            start = 0
            end = table_number
            
        col_preds = table_preds.split(separator)[start:end]
        
        for pred in col_preds:
            i+=1
            
            # Remove break lines
            if "\n" in pred:
                pred = pred.split('\n')[0].strip()
            # Remove commas
            if "," in pred:
                pred = pred.split(",")[0].strip()
            # Remove paranthesis
            if '(' in pred:
                pred = pred.split("(")[0].strip()
            #Remove points
            if '.' in pred:
                pred = pred.split(".")[0].strip()
            # Lower-case prediction
            pred = pred.strip().lower()
            
            if pred in text_to_label:
                predictions.append(text_to_label[pred])
            else:
                print(f"For test example {i} out of label space prediction: {pred}")
                predictions.append('-')
            
        # If more/less predictions for table
        if len(col_preds) < table_number:
            for m in range(0, table_number-len(col_preds)):
                predictions.append('-')
                i+=1

evaluation_old()

### Calculate Precision, Recall, Macro-F1 and Micro-F1

In [90]:
df = pd.read_csv("predictions/preds_gpt35_with_inst_2.csv",index_col=0)
labels, preds = df["label"], df["lionel_annot"]
# top_5 = df["top_5_preds"]
df.head()

,prompt_output_id,label,parsed_col_pred,class_pred_using_bert,highest_bertscore,top_5_preds,lionel_annot
0,0,telephone,phone number,telephone,[0.90746641 0.91230839 0.91649085 0.9181211 0...,"['Event/name', 'Date', 'faxNumber', 'priceRang...",telephone
1,0,Restaurant/name,restaurant name,Restaurant/name,[0.90765798 0.90860718 0.91736311 0.91822797 0...,"['Restaurant/description', 'Hotel/name', 'Rest...",Restaurant/name
2,0,postalCode,zip code,postalCode,[0.85932994 0.86107123 0.8723675 0.8726126 0...,"['Event/description', 'Duration', 'Event/name'...",postalCode
3,0,addressRegion,state,Country,[0.89595306 0.89995229 0.90346491 0.90814996 0...,"['Time', 'Duration', 'currency', 'Date', 'Coun...",addressRegion
4,0,Country,country,Country,[0.91340989 0.91425383 0.9207198 0.93141145 0...,"['Duration', 'currency', 'Time', 'Date', 'Coun...",Country


In [91]:
print(len(df))
print(len(df[df["label"] == df["class_pred_using_bert"]]))
df['label_in_top_5'] = df[['label','top_5_preds']].apply(
    lambda row: row['label'] in row['top_5_preds'], axis=1
)
print(len(df[df["label_in_top_5"] == True]))

250
140
197


In [58]:
def calculate_f1_scores(y_tests, y_preds, num_classes):
    types = list(set(labels))
    types = types + ["-"]
    
    y_tests = [types.index(y) for y in y_tests]
    y_preds = [types.index(y) for y in y_preds]
    
    #Confusion matrix
    cm = np.zeros(shape=(num_classes,num_classes))
    
    for i in range(len(y_tests)):
        cm[y_preds[i]][y_tests[i]] += 1
        
    report = {}
    
    for j in range(len(cm[0])):
        report[j] = {}
        report[j]['FN'] = 0
        report[j]['FP'] = 0
        report[j]['TP'] = cm[j][j]

        for i in range(len(cm)):
            if i != j:
                report[j]['FN'] += cm[i][j]
        for k in range(len(cm[0])):
            if k != j:
                report[j]['FP'] += cm[j][k]

        precision = report[j]['TP'] / (report[j]['TP'] + report[j]['FP'])
        recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])
        f1 = 2*precision*recall / (precision + recall)
        
        if np.isnan(f1):
            f1 = 0
        if np.isnan(precision):
            f1 = 0
        if np.isnan(recall):
            f1 = 0

        report[j]['p'] =  precision
        report[j]['r'] =  recall
        report[j]['f1'] = f1
    
    all_fn = 0
    all_tp = 0
    all_fp = 0

    for r in report:
        if r != num_classes-1:
            all_fn += report[r]['FN']
            all_tp += report[r]['TP']
            all_fp += report[r]['FP']
        
    class_f1s = [ report[class_]['f1'] for class_ in report]
    class_p = [ 0 if np.isnan(report[class_]['p']) else report[class_]['p'] for class_ in report]
    class_r = [ 0 if np.isnan(report[class_]['r']) else report[class_]['r'] for class_ in report]
    macro_f1 = sum(class_f1s[:-1]) / (num_classes-1)
    
    p =  sum(class_p[:-1]) / (num_classes-1)
    r =  sum(class_r[:-1]) / (num_classes-1)
    micro_f1 = all_tp / ( all_tp + (1/2 * (all_fp + all_fn) )) 
    
    per_class_eval = {}
    for index, t in enumerate(types[:-1]):
        per_class_eval[t] = {"Precision":class_p[index], "Recall": class_r[index], "F1": class_f1s[index]}
    
    evaluation = {
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Precision": p,
        "Recall": r
    }
    
    return evaluation, per_class_eval

In [92]:
evaluation, per_class_eval = calculate_f1_scores(labels, preds, 33)
print(evaluation)
print(per_class_eval)

{'Micro-F1': 0.8824742268041237, 'Macro-F1': 0.8407250484059463, 'Precision': 0.916870716089466, 'Recall': 0.8131448412698412}
{'EventStatusType': {'Precision': 1.0, 'Recall': 0.16666666666666666, 'F1': 0.2857142857142857}, 'Hotel/name': {'Precision': 1.0, 'Recall': 0.9, 'F1': 0.9473684210526316}, 'Review': {'Precision': 0.5, 'Recall': 0.25, 'F1': 0.3333333333333333}, 'EventAttendanceModeEnumeration': {'Precision': 0.8333333333333334, 'Recall': 0.7142857142857143, 'F1': 0.7692307692307692}, 'Event/description': {'Precision': 0.7272727272727273, 'Recall': 0.8888888888888888, 'F1': 0.7999999999999999}, 'Restaurant/name': {'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0}, 'Event/name': {'Precision': 1.0, 'Recall': 0.9, 'F1': 0.9473684210526316}, 'faxNumber': {'Precision': 1.0, 'Recall': 0.3333333333333333, 'F1': 0.5}, 'Country': {'Precision': 0.9166666666666666, 'Recall': 0.9166666666666666, 'F1': 0.9166666666666666}, 'DayOfWeek': {'Precision': 1.0, 'Recall': 0.75, 'F1': 0.8571428571428571}, '

/tmp/ipykernel_107517/4141412260.py:30: RuntimeWarning: invalid value encountered in double_scalars
  recall = report[j]['TP'] / (report[j]['TP'] + report[j]['FN'])


In [52]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, preds, average="micro")

(0.328, 0.328, 0.328, None)

## Error Analysis

In [ ]:
# "-" means the model replied with out of label or with I don't know
errors = 0
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        errors += 1
        print(f"Predicted as {predictions[i]} when it was {labels[i]}")
errors